...done using [Markdown Cheat Sheet](https://gtribello.github.io/mathNET/assets/notebook-writing.html) and [Unofficial Jupyter Extensions](https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/index.html)

# Imports and Installations

In [1]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
import custom_functions as func
import analysis_functions as afunc
import prep_data as prep
import plot_functions as plot

# Custom Functions

In [15]:
# get data as dataframe
def read_csv():
    df = pd.read_csv("C:/Users/Andrea/PycharmProjects/datascience/data/StackOverflow2020/survey_results_public.csv")  # df with answers
    schema = pd.read_csv("C:/Users/Andrea/PycharmProjects/datascience/data/StackOverflow2020/survey_results_schema.csv")  # df with questions
    return df, schema

In [16]:
# get description of a column (desc) as string by providing the column name (column_name)
def get_description(column_name, schema=schema):
    desc = schema.set_index('Column').loc[f'{column_name}']['QuestionText']
    return desc

In [17]:
# get the percentage of NaN values (perc_nan) and the number of columns with more than x percent of values missing (perc_nan_over_x) in df
def get_nan_perc(df, threshold):
    # columns with corresponding percentage of nan values
    perc_nan = df.isnull().sum()/len(df)
    # columns with more than x percent of nan values
    perc_nan_over_x = (df.isnull().sum()/len(df)) > threshold
    # number of columns
    number_over_x = np.sum(perc_nan > threshold)
    return perc_nan, perc_nan_over_x, number_over_x

In [18]:
# drop all missing values from a given subset or target column
def dropna_subset(df, subset):
    df = df.dropna(subset=[subset]).copy(deep=True)
    return df

In [19]:
def feature_overview(df, feature_column, dropna=False):
    val_counts = df[feature_column].value_counts(dropna=dropna)
    overview_df = pd.DataFrame({'count': val_counts, 'share': val_counts / val_counts.sum()})
    return overview_df

In [20]:
def feature_by_x(df, feature, x_column):
    # get feature counts per x_column & unstack multi-index
    counts = df.groupby(x_column)[feature].value_counts().unstack(level=0)
    # get feature percentages per x_column & unstack multi-index (get x by feature dataframe)
    shares = counts.apply(lambda x: x/x.sum())
    return counts, shares

In [21]:
def get_multiple_choice_answers():
    DevTypeAnswers = ["Academic researcher",
                    "Data or business analyst",
                    "Data scientist or machine learning specialist",
                    "Database administrator",
                    "Designer",
                    "Developer, back-end",
                    "Developer, desktop or enterprise applications",
                    "Developer, embedded applications or devices",
                    "Developer, front-end",
                    "Developer, full-stack",
                    "Developer, game or graphics",
                    "Developer, mobile",
                    "Developer, QA or test",
                    "DevOps specialist",
                    "Educator",
                    "Engineer, data",
                    "Engineer, site reliability",
                    "Engineering manager",
                    "Marketing or sales professional",
                    "Product manager",
                    "Scientist",
                    "Senior Executive (C-Suite, VP, etc.)",
                    "System administrator"]
    NEWJobHuntAnswers = ['Just because',
                        'Having a bad day (or week or month) at work',
                        'Wanting to share accomplishments with a wider network',
                        'Curious about other opportunities',
                        'Better compensation',
                        'Trouble with my teammates',
                        'Trouble with my direct manager',
                        'Trouble with leadership at my company',
                        'Better work/life balance',
                        'Wanting to work with new technologies',
                        'Growth or leadership opportunities',
                        'Looking to relocate']
    JobFactorsAnswers = ['Remote work options',
                        'Office environment or company culture',
                        'Financial performance or funding status of the company or organization',
                        'Opportunities for professional development',
                        'Diversity of the company or organization',
                        'How widely used or impactful my work output would be',
                        'Industry that I’d be working in',
                        'Specific department or team I’d be working on',
                        'Flex time or a flexible schedule',
                        'Languages, frameworks, and other technologies I’d be working with',
                        'Family friendliness']
    return DevTypeAnswers, NEWJobHuntAnswers, JobFactorsAnswers

In [22]:
def dummy_multiple_choice(df, column, answer_list):
    for x in answer_list:
        sername = column + '_' + str(answer_list.index(x))
        ser = df[column].str.contains(x).rename(sername).to_frame()*1
        df = pd.concat([df, ser], axis=1)
    return df

In [23]:
def get_multiple_choice(df, column_regex, answerlist_mc):
    df_mc = df.filter(regex=column_regex, axis=1).dropna().sum().rename('count')
    share = df_mc / df.shape[0]
    df_mc = pd.concat\
        ([df_mc, share.rename('share'), pd.Series(answerlist_mc, index=df_mc.index, name='answers')], axis=1)\
        .set_index('answers')
    return df_mc

In [24]:
def new_happy_bins(df):
    # make 3 new categories for categories column (excl. NaN)
    df['Job_Satisfaction_bins'] = df['Job_Satisfaction'].replace({'Very satisfied': 'satisfied',
                                              'Slightly satisfied': 'satisfied',
                                              'Very dissatisfied': 'dissatisfied',
                                              'Slightly dissatisfied': 'dissatisfied',
                                              'Neither satisfied nor dissatisfied': 'neither'})
    happy_index = ['Very satisfied', 'Slightly satisfied', 'Neither satisfied nor dissatisfied',
                   'Slightly dissatisfied', 'Very dissatisfied', 'not answered']
    happy_index_bins = ['satisfied', 'dissatisfied', 'neither']
    return df, happy_index, happy_index_bins

# Load & View Data

In [26]:
df_raw, schema = func.read_csv()
df = df_raw.copy(deep=True)
df.head()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,...,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,...,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,...,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,...,NaN,NaN,No,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat less welcome now than last year,40.0,7,4
4,5,"I used to be a developer by profession, but no...",Yes,31.0,16,NaN,NaN,NaN,United States,NaN,...,Easy,Too short,No,"Computer science, computer engineering, or sof...",Django;Ruby on Rails,Ruby on Rails,Just as welcome now as I felt last year,NaN,15,8


In [27]:
#### get an overview of the dataframe ####
# get column descriptions (respective survey questions)
survey_question = get_description('MainBranch')
# get dataframe dimensions
shape = df.shape
# overview over nan values
perc_nan, perc_nan_over_x, number_over_x = get_nan_perc(df, .5)

# Data Preparation

In [28]:
#### rename some colums for better readability later
df = df.rename(columns={"PurchaseWhat": "Influence_On_Purchases", "JobSat": "Job_Satisfaction"})

#### create salary bins (yearly salary in USD)
df['Salary_Group'] = pd.cut(df['ConvertedComp'], bins=[0, df.ConvertedComp.median(), 2000000], labels=["below median", "above median"])

#### add dummies for multiple choice columns to extract number of individual mentions
DevTypeAnswers, NEWJobHuntAnswers, JobFactorsAnswers = func.get_multiple_choice_answers()
df = dummy_multiple_choice(df, 'DevType', DevTypeAnswers)
df = dummy_multiple_choice(df, 'NEWJobHunt', NEWJobHuntAnswers)
df = dummy_multiple_choice(df, 'JobFactors', JobFactorsAnswers)

df_full = df.copy(deep=True)
#### drop all missing values from target column: "job satisfaction"
df = prep.dropna_subset(df, 'Job_Satisfaction')
# define a fixed index order for "Job Satisfaction"
happy_index = ['Very satisfied', 
               'Slightly satisfied', 
               'Neither satisfied nor dissatisfied',
               'Slightly dissatisfied', 
               'Very dissatisfied', 
               'not answered']

C:\Users\Andrea\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


# ...the Real Questions